In [2]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
sim = Aer.get_backend('aer_simulator')

In [3]:
def XOR(qc, a, b, output):
    qc.cx(a, output)
    qc.cx(b, output)

In [33]:
clause_list = [[0,1],
               [0,2],
               [1,3],
               [2,3]]

In [34]:
in_qubits = QuantumRegister(4, name='v')
clause_qubits = QuantumRegister(4, name='c')
out0 = QuantumRegister(1, name='out')
outputBits = ClassicalRegister(4, 'cbits')
qc = QuantumCircuit(in_qubits, clause_qubits, out0, outputBits)

def sudoku_oracle(qc, clause_list, clause_qubits):
    # compute
    i = 0
    for clause in clause_list:
        XOR(qc, in_qubits[clause[0]], in_qubits[clause[1]], clause_qubits[i])
        i += 1
    
    qc.mct(clause_qubits, out0)
    
    # uncompute
    i = 0
    for clause in clause_list:
        XOR(qc, in_qubits[clause[0]], in_qubits[clause[1]], clause_qubits[i])
        i += 1
# initiliaze out0 in |->        
qc.initialize([1, -1]/np.sqrt(2), out0)

# put everything in a superposition
qc.h(in_qubits)  
qc.barrier() 
sudoku_oracle(qc, clause_list, clause_qubits)
# qc.barrier() 
# qc.append(diffuser(4), [0,1,2,3])
# sudoku_oracle(qc, clause_list, clause_qubits)
# qc.barrier() 
# diffuser(qc, in_qubits, 4)
# qc.measure(in_qubits, outputBits)

qc.draw(fold=-1)

┌───┐               ░                                                                                      
    v_0: ─────────────┤ H ├───────────────░───■─────────■──────────────────────────────────■─────────■───────────────────────────
                      ├───┤               ░   │         │                                  │         │                           
    v_1: ─────────────┤ H ├───────────────░───┼────■────┼─────────■────────────────────────┼────■────┼─────────■─────────────────
                      ├───┤               ░   │    │    │         │                        │    │    │         │                 
    v_2: ─────────────┤ H ├───────────────░───┼────┼────┼────■────┼─────────■──────────────┼────┼────┼────■────┼─────────■───────
                      ├───┤               ░   │    │    │    │    │         │              │    │    │    │    │         │       
    v_3: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────■────┼────■─────────┼────┼────┼────┼────┼────■────┼────■──
                      └───┘               ░ ┌─┴─┐┌─┴─┐  │    │    │    │    │    │       ┌─┴─┐┌─┴─┐  │    │    │    │    │    │  
    c_0: ─────────────────────────────────░─┤ X ├┤ X ├──┼────┼────┼────┼────┼────┼────■──┤ X ├┤ X ├──┼────┼────┼────┼────┼────┼──
                                          ░ └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │    │    │  └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │    │  
    c_1: ─────────────────────────────────░───────────┤ X ├┤ X ├──┼────┼────┼────┼────■────────────┤ X ├┤ X ├──┼────┼────┼────┼──
                                          ░           └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │            └───┘└───┘┌─┴─┐┌─┴─┐  │    │  
    c_2: ─────────────────────────────────░─────────────────────┤ X ├┤ X ├──┼────┼────■──────────────────────┤ X ├┤ X ├──┼────┼──
                                          ░                     └───┘└───┘┌─┴─┐┌─┴─┐  │                      └───┘└───┘┌─┴─┐┌─┴─┐
    c_3: ─────────────────────────────────░───────────────────────────────┤ X ├┤ X ├──■────────────────────────────────┤ X ├┤ X ├
         ┌──────────────────────────────┐ ░                               └───┘└───┘┌─┴─┐                              └───┘└───┘
    out: ┤ Initialize(0.70711,-0.70711) ├─░─────────────────────────────────────────┤ X ├────────────────────────────────────────
         └──────────────────────────────┘ ░                                         └───┘                                        
cbits: 4/════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [28]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

def diffuser(qc,a,nqubits):
#     qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(a[qubit])
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(a[qubit])
    # Do multi-controlled-Z gate
    qc.h(a[nqubits-1])
    qc.mct([a[i] for i in range(nqubits-1)], a[nqubits-1])  # multi-controlled-toffoli
    qc.h(a[nqubits-1])
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(a[qubit])
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(a[qubit])

In [35]:
# simulator = Aer.get_backend('aer_simulator')
# qc = transpile(qc, simulator)

# # Run and get counts
# result = simulator.run(qc).result()
# counts = result.get_counts(qc)
# plot_histogram(counts, title='Bell-State counts')

# Measure
aer_sim = Aer.get_backend('aer_simulator')
# transpiled_grover_circuit = transpile(qc, aer_sim)
# transpiled_grover_circuit.save_statevector()
qc.save_statevector()
qobj = assemble(qc)
result = aer_sim.run(qobj).result()
statevec = result.get_statevector()
# partial_density_matrix = partial_trace(statevec, [1, 2])

# extract the statevector out of the density matrix
# partial_statevector = np.diagonal(partial_density_matrix)
# print(partial_statevector)
from qiskit_textbook.tools import vector2latex
vector2latex(statevec, pretext="|\\psi\\rangle =")

<IPython.core.display.Math object>

In [33]:
var_qubits = QuantumRegister(4, name='v')
clause_qubits = QuantumRegister(4, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(4, name='cbits')
qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

# Initialize 'out0' in state |->
qc.initialize([1, -1]/np.sqrt(2), output_qubit)

# Initialize qubits in state |s>
qc.h(var_qubits)
qc.barrier()  # for visual separation

## First Iteration
# Apply our oracle
sudoku_oracle(qc, clause_list, clause_qubits)
qc.barrier()  # for visual separation
# Apply our diffuser
qc.append(diffuser(4), [0,1,2,3])

## Second Iteration
sudoku_oracle(qc, clause_list, clause_qubits)
qc.barrier()  # for visual separation
# Apply our diffuser
qc.append(diffuser(4), [0,1,2,3])

# Measure the variable qubits
qc.measure(var_qubits, cbits)

qc.draw(fold=-1)

┌───┐               ░                                                                                       ░ ┌────────┐                                                                                      ░ ┌────────┐┌─┐         
    v_0: ─────────────┤ H ├───────────────░───■─────────■──────────────────────────────────■─────────■────────────────────────────░─┤0       ├──■─────────■──────────────────────────────────■─────────■────────────────────────────░─┤0       ├┤M├─────────
                      ├───┤               ░   │         │                                  │         │                            ░ │        │  │         │                                  │         │                            ░ │        │└╥┘┌─┐      
    v_1: ─────────────┤ H ├───────────────░───┼────■────┼─────────■────────────────────────┼────■────┼─────────■──────────────────░─┤1       ├──┼────■────┼─────────■────────────────────────┼────■────┼─────────■──────────────────░─┤1       ├─╫─┤M├──────
                      ├───┤               ░   │    │    │         │                        │    │    │         │                  ░ │  U$_s$ │  │    │    │         │                        │    │    │         │                  ░ │  U$_s$ │ ║ └╥┘┌─┐   
    v_2: ─────────────┤ H ├───────────────░───┼────┼────┼────■────┼─────────■──────────────┼────┼────┼────■────┼─────────■────────░─┤2       ├──┼────┼────┼────■────┼─────────■──────────────┼────┼────┼────■────┼─────────■────────░─┤2       ├─╫──╫─┤M├───
                      ├───┤               ░   │    │    │    │    │         │              │    │    │    │    │         │        ░ │        │  │    │    │    │    │         │              │    │    │    │    │         │        ░ │        │ ║  ║ └╥┘┌─┐
    v_3: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────■────┼────■─────────┼────┼────┼────┼────┼────■────┼────■───░─┤3       ├──┼────┼────┼────┼────┼────■────┼────■─────────┼────┼────┼────┼────┼────■────┼────■───░─┤3       ├─╫──╫──╫─┤M├
                      └───┘               ░ ┌─┴─┐┌─┴─┐  │    │    │    │    │    │       ┌─┴─┐┌─┴─┐  │    │    │    │    │    │   ░ └────────┘┌─┴─┐┌─┴─┐  │    │    │    │    │    │       ┌─┴─┐┌─┴─┐  │    │    │    │    │    │   ░ └────────┘ ║  ║  ║ └╥┘
    c_0: ─────────────────────────────────░─┤ X ├┤ X ├──┼────┼────┼────┼────┼────┼────■──┤ X ├┤ X ├──┼────┼────┼────┼────┼────┼───░───────────┤ X ├┤ X ├──┼────┼────┼────┼────┼────┼────■──┤ X ├┤ X ├──┼────┼────┼────┼────┼────┼───░────────────╫──╫──╫──╫─
                                          ░ └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │    │    │  └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │    │   ░           └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │    │    │  └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │    │   ░            ║  ║  ║  ║ 
    c_1: ─────────────────────────────────░───────────┤ X ├┤ X ├──┼────┼────┼────┼────■────────────┤ X ├┤ X ├──┼────┼────┼────┼───░─────────────────────┤ X ├┤ X ├──┼────┼────┼────┼────■────────────┤ X ├┤ X ├──┼────┼────┼────┼───░────────────╫──╫──╫──╫─
                                          ░           └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │            └───┘└───┘┌─┴─┐┌─┴─┐  │    │   ░                     └───┘└───┘┌─┴─┐┌─┴─┐  │    │    │            └───┘└───┘┌─┴─┐┌─┴─┐  │    │   ░            ║  ║  ║  ║ 
    c_2: ─────────────────────────────────░─────────────────────┤ X ├┤ X ├──┼────┼────■──────────────────────┤ X ├┤ X ├──┼────┼───░───────────────────────────────┤ X ├┤ X ├──┼────┼────■──────────────────────┤ X ├┤ X ├──┼────┼───░────────────╫──╫──╫──╫─
                                          ░                     └───┘└───┘┌─┴─┐┌─┴─┐  │                      └───┘└───┘┌─┴─┐┌─┴─┐ ░                               └───┘└───┘┌─┴─┐┌─┴─┐  │                      └───┘└───┘┌─┴─┐┌─┴─┐ ░            ║  ║  ║  ║ 
    c_3: ─────────────────────────────────░───────────────────────────────┤ X ├┤ X ├──■────────────────────────────────┤ X ├┤ X ├─░─────────────────────────────────────────┤ X ├┤ X ├──■────────────────────────────────┤ X ├┤ X ├

In [39]:
# attempt at a 3x3 sudoku

clause_list = [
               [0, 1, 2],
               [3, 4, 5],
               [6, 7, 8],
               [0, 3, 6],
               [1, 4, 7],
               [2, 5, 8]
              ]


in_qubits = QuantumRegister(18, name='v')
clause_qubits = QuantumRegister(12, name='c')
out0 = QuantumRegister(1, name='out')
outputBits = ClassicalRegister(18, 'cbits')
qc = QuantumCircuit(in_qubits, clause_qubits, out0, outputBits)

def sudoku_oracle(qc, clause_list, clause_qubits):
    # compute
    i = 0
    for clause in clause_list:
        XOR3(qc, in_qubits[2*clause[0]:2*clause[0]+2], in_qubits[2*clause[1]:2*clause[1]+2], in_qubits[2*clause[2]:2*clause[2]+2], clause_qubits[2*i:2*i+2])
        i += 1
    
    qc.mct(clause_qubits, out0)
    
    # uncompute
    i = 0
    for clause in clause_list:
        XOR3(qc, in_qubits[2*clause[0]:2*clause[0]+2], in_qubits[2*clause[1]:2*clause[1]+2], in_qubits[2*clause[2]:2*clause[2]+2], clause_qubits[2*i:2*i+2])
        i += 1
# initiliaze out0 in |->        
qc.initialize([1, -1]/np.sqrt(2), out0)

# put everything in a superposition
qc.h(in_qubits)  
qc.barrier() 
sudoku_oracle(qc, clause_list, clause_qubits)
qc.barrier() 
qc.append(diffuser(18), range(0,18))
sudoku_oracle(qc, clause_list, clause_qubits)
qc.barrier() 
qc.append(diffuser(18), range(0,18))
qc.measure(in_qubits, outputBits)

qc.draw(fold=-1)

┌───┐               ░                                                                                                                                                                                                                                                                                                                                                                               ░ ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                              ░ ┌─────────┐┌─┐                                                   
     v_0: ─────────────┤ H ├───────────────░───■───────────────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤0        ├──■───────────────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤0        ├┤M├───────────────────────────────────────────────────
                       ├───┤               ░   │                                                           │                                                                                                                            │                                                           │                                                                                                                      ░ │         │  │                                                           │                                                                                                                            │                                                           │                                                                                                                      ░ │         │└╥┘┌─┐                                                
     v_1: ─────────────┤ H ├───────────────░───┼────■──────────────────────────────────────────────────────┼────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────■──────────────────────────────────────────────────────┼────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤1        ├──┼────■──────────────────────────────────────────────────────┼────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────■──────────────────────────────────────────────────────┼────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤1        ├─╫─┤M├────────────────────────────────────────────────
                       ├───┤               ░   │    │                                                      │    │                                                                                                                       │    │                                                      │    │                                                                                                                 ░ │         │  │    │                                                      │    │                                                                                                   

In [8]:
# We will use separate registers to name the bits
in_qubits = QuantumRegister(6, name='input')
out_qubit = QuantumRegister(2, name='output')
qc = QuantumCircuit(in_qubits, out_qubit)
XOR3(qc, in_qubits[0:2], in_qubits[2:4], in_qubits[4:6], out_qubit)
qc.draw()

input_0: ──■───────────────────────────
            │                           
 input_1: ──┼────■──────────────────────
            │    │                      
 input_2: ──┼────┼────■─────────────────
            │    │    │                 
 input_3: ──┼────┼────┼────■────────────
            │    │    │    │            
 input_4: ──┼────┼────┼────┼────■───────
            │    │    │    │    │       
 input_5: ──┼────┼────┼────┼────┼────■──
          ┌─┴─┐  │  ┌─┴─┐  │  ┌─┴─┐  │  
output_0: ┤ X ├──┼──┤ X ├──┼──┤ X ├──┼──
          └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐
output_1: ─────┤ X ├─────┤ X ├─────┤ X ├
               └───┘     └───┘     └───┘

In [2]:
def XOR3(qc, a, b, c, output):
    qc.cx(a[0], output[0])
    qc.cx(a[1], output[1])
    qc.cx(b[0], output[0])
    qc.cx(b[1], output[1])
    qc.cx(c[0], output[0])
    qc.cx(c[1], output[1])
    